In [1]:
"""
Extract gridMET variables 
Google Earth Engine (GEE) Python API
Author: maxwell.cook@colorado.edu
"""

import os, sys
import ee
import geemap
import time

# Custom functions
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *

ee.Authenticate()
ee.Initialize(project='jfsp-aspen')

print("Success !")

Success !


In [2]:
# Load gridded FRP data
grid = ee.FeatureCollection('projects/jfsp-aspen/assets/viirs_snpp_jpss1_afd_latlon_aspenfires_pixar_gridstats_')
print(f"Number of aspen fires: {grid.size().getInfo()}")
print(grid.first().propertyNames().getInfo())

Number of aspen fires: 49022
['Fire_Year', 'grid_index', 'Fire_ID', 'max_date', 'first_obs', 'afd_count', 'Ig_Date', 'Last_Date', 'system:index', 'last_obs']


In [3]:
# get a list of unique active fire days
fire_days = grid.aggregate_array('first_obs').getInfo()
fire_days = set(fire_days)
print(f"Number of unique 'fire days': {len(fire_days)}")

# get the min and max DOY
fire_days_doy = [datetime.strptime(day, "%Y-%m-%d").timetuple().tm_yday for day in fire_days]
doy_min = min(fire_days_doy)
doy_max = max(fire_days_doy)
print(f"Fire DOY range: {doy_min} to {doy_max}")

Number of unique 'fire days': 382
Fire DOY range: 89 to 317


In [ ]:
# Load the gridmet image collection

In [5]:
gridmet = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET')
print(f"gridMET bands available for analysis:\n\n{gridmet.first().bandNames().getInfo()}\n")

gridMET bands available for analysis:

['pr', 'rmax', 'rmin', 'sph', 'srad', 'th', 'tmmn', 'tmmx', 'vs', 'erc', 'eto', 'bi', 'fm100', 'fm1000', 'etr', 'vpd']



In [11]:
# select our variables of interest:
# 1. Vapor Pressure Deficit and 2. Energy Release Component
gridmet = gridmet.select(['vpd','erc'])
gridmet = gridmet.filter(ee.Filter.calendarRange(2018,2023,'year')) # filter to our fire years
print(gridmet.first().bandNames().getInfo())

['vpd', 'erc']


In [12]:
# calculate the 15 year average VP, ERC and deviation from it
mean15 = gridmet.filter(ee.Filter.And(
    ee.Filter.calendarRange(doy_min,doy_max,'day_of_year'),
    ee.Filter.calendarRange(2008,2023,'year'))).mean() 
gridmet.first().bandNames().getInfo()

attrs = ee.List(gridmet.first().bandNames()) # grab the band names for renaming
attrs = attrs.map(lambda atr: ee.String(atr).cat('_dv'))
print(attrs.getInfo())

# function to subtract the mean from each daily image
def subtract_anomaly(image):
    anomaly = image.subtract(mean15).rename(attrs)
    return image.addBands(anomaly)

# apply across the gridmet data
gridmet_ = gridmet.map(subtract_anomaly)
gridmet_.first()

['vpd_dv', 'erc_dv']


In [21]:
# Check the type of 'first_obs' in the grid
print(grid.first().get('first_obs').getInfo())

2018-06-01


In [25]:
# map over the list of unique dates
def process_gridcell(gridcell):
    """ calculate the daily gridmet for active fire detections """
   
    # get gridcell attributes
    gridIDX = gridcell.get('grid_index') # grab the fire id
    end_date = ee.Date(gridcell.get('max_date')).advance(1, 'days') # day of maximum FRP
    start_date =end_date.advance(-1, 'days') # day before maximum FRP

    # filter the gridmet
    gridmet_cell = gridmet_.filterDate(start_date, end_date).mean()

    gridcell_stats = gridmet_cell.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=gridcell.geometry(),
        scale=375 # ensures gricells are included
    )

    return ee.Feature(None, gridcell_stats.set('grid_index', gridIDX))

# map the function across gridcells
results = grid.map(process_gridcell)
results.limit(10)

In [26]:
sample = results.limit(10).getInfo()
props = [f['properties'] for f in sample['features']]
df = pd.DataFrame(props)
df.head()

,erc,erc_dv,grid_index,vpd,vpd_dv
0,86.0,27.418488,919906,1.51,0.334716
1,86.0,27.418488,919907,1.51,0.334716
2,86.0,27.418488,919908,1.51,0.334716
3,87.0,27.202328,922166,1.61,0.345386
4,86.0,27.418488,922171,1.51,0.334716


In [ ]:
# Export the table. 
export_task = ee.batch.Export.table.toDrive(
    collection=results,
    description='gridstats_gridmet',
    fileNamePrefix='gridstats_gridmet',
    fileFormat='CSV', 
    folder='GRIDMET'
)

export_task.start() # Start the export task
print("Export to Earth Engine Asset started!")
# Monitor the task until it's finished
monitor_export(export_task, 120) 

Export to Earth Engine Asset started!
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting fo